In [5]:
import os
from dotenv import load_dotenv
from huggingface_hub import InferenceClient
import speech_recognition as sr
import keyboard  # 키보드 입력 감지 모듈
import io
import pygame
from gtts import gTTS





load_dotenv()

True

In [3]:
HF_TOKENS = os.getenv("HF_TOKENS")


In [4]:

api_key = ''
client = InferenceClient(api_key=HF_TOKENS)
messages = [
    {
        "role": "user",
        "content": "What is the capital of France?"
    }
]
completion = client.chat.completions.create(
    model="meta-llama/Llama-3.2-1B-Instruct",
    messages=messages,
    max_tokens=500
)

print(completion.choices[0].message)

ChatCompletionOutputMessage(role='assistant', content='The capital of France is Paris.', tool_calls=None)


In [6]:

text_result = []  # 음성 인식 결과를 담을 리스트


def recognize_speech():
    # Recognizer 객체 생성
    r = sr.Recognizer()

    print("스페이스 바를 눌러 음성 녹음을 시작/중지하세요. ESC 키로 프로그램을 종료합니다.")

    recording = False  # 녹음 상태를 확인하는 플래그

    with sr.Microphone() as source:
        print("환경 소음 조정 중...")
        r.adjust_for_ambient_noise(source, duration=0.2)  # 주변 소음 조정
        print("소음 조정 완료.")

        while True:
            try:
                # ESC 키로 프로그램 종료
                if keyboard.is_pressed("esc"):
                    print("ESC 키가 눌렸습니다. 프로그램을 종료합니다.")
                    break

                # 스페이스 바로 녹음 시작/중지
                if keyboard.is_pressed("space"):
                    if not recording:
                        print("녹음을 시작합니다. 다시 스페이스 바를 누르면 녹음이 중지됩니다.")
                        recording = True
                        user_audio = r.listen(source)  # 사용자 음성 입력
                        print("녹음이 중지되었습니다. 음성을 처리 중입니다...")
                        recording = False

                        # 음성 -> 텍스트 변환 (한국어 설정)
                        text = r.recognize_google(user_audio, language='ko-KR')
                        print("인식된 텍스트: ", text)
                        text_result.append(text)  # 결과를 리스트에 추가

                        print("현재까지 인식된 텍스트:", text_result)

                        # 스페이스 바 해제 대기 (다시 누를 수 있도록)
                        while keyboard.is_pressed("space"):
                            pass

            except sr.RequestError as e:
                print(f"API 요청 실패: {e}")

            except sr.UnknownValueError:
                print("음성을 인식할 수 없습니다. 다시 시도하세요.")

    return text_result  # 최종 결과 반환


if __name__ == "__main__":
    result = recognize_speech()  # 결과를 변수에 담아서 실행
    print("최종 음성 인식 결과:", result)


스페이스 바를 눌러 음성 녹음을 시작/중지하세요. ESC 키로 프로그램을 종료합니다.
환경 소음 조정 중...
소음 조정 완료.
녹음을 시작합니다. 다시 스페이스 바를 누르면 녹음이 중지됩니다.
녹음이 중지되었습니다. 음성을 처리 중입니다...
인식된 텍스트:  내 이름은 권용희
현재까지 인식된 텍스트: ['내 이름은 권용희']
ESC 키가 눌렸습니다. 프로그램을 종료합니다.
최종 음성 인식 결과: ['내 이름은 권용희']


In [8]:

messages = [{"role": "user", "content": "Hello, how are you?"}]



# user_input = input("User: ")
user_input = text_result[0]  # 리스트의 첫 번째 요소를 가져옴


if user_input.lower() in ['exit', 'quit', 'bye', 'stop', 'escape']:
    print("Bye!")


messages.append({"role": "user", "content": user_input})

completion = client.chat.completions.create(
    model="meta-llama/Llama-3.2-1B-Instruct",
    messages=messages,
    max_tokens=500
)

bot_reply = completion.choices[0].message['content']
print(f"Bot: {bot_reply}")

messages.append({"role": "assistant", "content": bot_reply})


Bot: hey 권용희 prostitutes greetings. I'm an AI, so we can talk about anything you want. How was your week?


In [10]:


# gTTS 객체 생성 및 메모리에 저장
def get_tts_audio(text, lang='en'):
    tts = gTTS(text, lang=lang)
    audio_stream = io.BytesIO()
    tts.write_to_fp(audio_stream)
    audio_stream.seek(0)  # 스트림의 시작으로 이동
    return audio_stream

# 오디오 재생 함수
def play_audio(audio_stream):
    pygame.mixer.init()
    pygame.mixer.music.load(audio_stream, "mp3")
    pygame.mixer.music.play()

    # 재생이 끝날 때까지 대기
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

# 메인 코드
if __name__ == "__main__":
    audio_stream = get_tts_audio(bot_reply)
    play_audio(audio_stream)
